In [2]:
import pandas as pd
import brainlit
from brainlit.utils import *
import csv
import pandas as pd
import numpy as np
import pickle
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from statsmodels.discrete.discrete_model import Logit
from sklearn.metrics import plot_roc_curve
from sklearn import datasets, metrics, model_selection

from keras.models import Sequential
from keras.layers import Dense,  Activation
from keras.regularizers import L1L2
from keras.utils import plot_model

from sklearn.preprocessing import LabelEncoder
from scipy import stats
import seaborn as sns

Using TensorFlow backend.


In [3]:
url = "s3://mouse-light-viz/precomputed_volumes/brain2"
ngl_skel = NeuroglancerSession(url+"_segments", mip=0)
working_ids = []
for seg_id in range(300):
    try:
        segment = ngl_skel.cv.skeleton.get(seg_id)
        working_ids.append(seg_id)
    except:
          pass

print (working_ids)

Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 41.39it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 39.47it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 49.00it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 37.79it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 40.78it/s]

[1, 2, 4, 5, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 20, 21, 23, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]


In [4]:
segment = ngl_skel.cv.skeleton.get(2)
ngl = NeuroglancerSession(url,mip=0)
voxel = ngl_skel._get_voxel(2, 1)
offset = np.array([15,15,15])
v_off = voxel + offset
bounds = Bbox(voxel,voxel+np.array([1,1,1]))
b_off = bounds - [15,15,15]
print(bounds - [15,15,15] , bounds)
img = ngl.pull_bounds_img(bounds)
img1 = ngl.pull_bounds_img(b_off)
img,img1

Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.30it/s]


Bbox([12773, 12597, 4436],[12774, 12598, 4437], dtype=int32) Bbox([12788, 12612, 4451],[12789, 12613, 4452], dtype=int32)


(array(15022, dtype=uint16), array(14347, dtype=uint16))

In [15]:
output_feats = 'intensity_mip0_trial1.csv'
int_verts = pd.DataFrame()
flag = 'w';
ids = working_ids
print (working_ids)

[1, 2, 4, 5, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 20, 21, 23, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]


In [16]:
ids = working_ids
sel_ids=[]
for i in range(1,len(ids),3):
    sel_ids.append(ids[i])
sel_ids

[2, 7, 11, 14, 17, 23, 33, 37, 40, 43, 47, 51, 54, 57, 60, 63, 66]

In [17]:
df = pd.DataFrame()
with open(output_feats,flag) as f:
    for seg_id in [2,7,11,14,17]:
        segment = ngl_skel.cv.skeleton.get(seg_id)
        for v_id,vertex in enumerate(segment.vertices):
            ngl = NeuroglancerSession(url,mip=0)
            try:
                if(v_id<100):
                    voxel = ngl_skel._get_voxel(seg_id, v_id)
                    offset = np.array([15,15,15])
                    bounds = Bbox(voxel,voxel+np.array([1,1,1]))
                    img = ngl.pull_bounds_img(bounds)
                    #print(img)
                    v_off = voxel + offset
                    #bounds_off = Bbox(v_off,v_off+np.array([1,1,1]))
                    bounds_off = bounds + [10,15,15]
                    img_off = ngl.pull_bounds_img(bounds_off)
                    df = df.append(
                                {
                                    "Segment": int(seg_id),
                                    "Vertex": int(v_id),
                                    "Label": 1,
                                    "Features": img,
                                },
                                ignore_index=True,
                            )

                    df = df.append(
                                {
                                    "Segment": int(seg_id),
                                    "Vertex": int(v_id),
                                    "Label": 0,
                                    "Features": img_off,
                                },
                                ignore_index=True,
                            )

                    t0 = df['Features'][0]
                    t0 = t0.transpose()
                    t0 = np.append(t0,int(df['Label'][0]))
                    t1 = df['Features'][1]
                    t1 = t1.transpose()
                    t1 = np.append(t1,int(df['Label'][1]))
                    #print(t0)
                    writer = csv.writer(f)
                    writer.writerows([t0])
                    writer.writerows([t1])
                    #print(df)
                    df = pd.DataFrame()
                else:
                    break
            except:
                pass

Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 33.35it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 21.32it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 66.06it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 31.68it/s]


In [28]:
df

""
